**Statistical tests**

This notebook is a playground for showing and testing statistical methods used by the `gpsea` library 
to discover genotype-phenotype correlations in patients annotated with HPO terms.

# Install the `gpsea` library

The notebook needs `gpsea` to be installed in the Python environment. `gpsea` is still mostly work in progress, so the best way is to install from sources.

We assume availability of `git` and Python 3.8 or better in the environment.

```shell
git clone git@github.com:monarch-initiative/gpsea.git
cd gpsea
git checkout develop && git pull

python3 -m pip install --editable .[test]

pytest
```

First, we download the source code from the GitHub repository, and we switch to `develop` branch to access the bleeding-edge features. Then, we install `gpsea` into the active environment, including the [dependencies](https://github.com/monarch-initiative/gpsea/blob/020832e850ef7107aa9de61462bc3490e0deb574/pyproject.toml#L34). As an optional last step, we run the tests to ensure installation went well.

With this setup, we are ready to run the rest of the notebook, assuming the notebook kernel corresponds to the Python environment where `gpsea` was installed to.

# Example analysis

For the purpose of this analysis, we use a cohort of patients with mutations in *SUOX* gene, curated from published case reports. 

The patient data includes:
- a `str` identifier that is unique to the patient within the cohort,
- a list of clinical signs and symptoms encoded into terms of Human Phenotype Ontology,
- one or more causal mutations in *SUOX* gene

The patient data is formatted as phenopackets of phenopacket schema.

## Create patient cohort

Let's start with loading of the patient data from the phenopackets:

In [1]:
import hpotk

from gpsea.preprocessing import configure_caching_cohort_creator

hpo_url = 'https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2023-10-09/hp.json'
hpo = hpotk.load_minimal_ontology(hpo_url)

cohort_creator = configure_caching_cohort_creator(hpo)

For reproducibility, we download a specific HPO version `v2023-10-09` and we will use it to load the phenopacket cohort in the cohort creator.

Now we can ETL the phenopackets into data format expected by `gpsea`. 

Assuming the notebook is run from its location within `gpsea` repository, we use the pe   t h e   located at [notebooks/SUOX/phenopackets](https://github.com/monarch-initiative/gpsea/tree/develop/notebooks/SUOX/phenopackets):

In [2]:
import os
import sys
from gpsea.preprocessing import load_phenopacket_folder

fpath_parent = os.path.dirname(os.getcwd())
fpath_phenopacket_dir = os.path.join(fpath_parent, 'notebooks', 'SUOX', 'phenopackets')
print(f'Loading phenopackets from {fpath_phenopacket_dir}', file=sys.stderr)

cohort = load_phenopacket_folder(fpath_phenopacket_dir, cohort_creator)

Loading phenopackets from /home/ielis/ielis/phenotypes/gpsea/notebooks/SUOX/phenopackets
Patients Created: 100%|██████████| 35/35 [00:00<00:00, 620.92it/s]
Validated under none policy
No errors or warnings were found


The code finds the directory with phenopackets and starts the loading.

The loading extracts the identifiers, HPO terms and vairants from the phenopacket JSON files and proceeds with functional annotation of the variants. 

> We use Variant Effect Predictor (VEP) REST endpoint to perform the annotation. However, we cache the results to conserve bandwidth and to speed up the subsequent runs.

The loading is concluded with validation of the HPO terms and variants.

## Explore the cohort

Let's explore the cohort to gain preliminary insights.

In [3]:
from IPython.display import HTML, display
from gpsea.view import CohortViewer

viewer = CohortViewer(hpo)

Show the number of cohort members, count of unique HPO terms and variants:

In [4]:
display(HTML(viewer.cohort_summary_table(cohort)))

Item,Description
Total Individuals,35
Total Unique HPO Terms,32
Total Unique Variants,48


Count the number of times an HPO term is used as an annotation in the cohort member:

In [5]:
display(HTML(viewer.hpo_term_counts_table(cohort))) ## Add Labels to output

HPO Term,Count
Seizure (HP:0001250),28
Hypotonia (HP:0001252),15
Sulfocysteinuria (HP:0032350),13
Abnormality of extrapyramidal motor function (HP:0002071),11
Hypertonia (HP:0001276),11
Microcephaly (HP:0000252),10
Neurodevelopmental delay (HP:0012758),8
Ectopia lentis (HP:0001083),7
Hypocystinemia (HP:0500152),7
Hypouricemia (HP:0003537),7


We will work with the most clinically relevant *SUOX* transcript:

In [6]:
tx_id = 'NM_001032386.2'

In [7]:
display(HTML(viewer.variants_table(cohort, tx_id))) 

Variant,Effect,Count,Key
c.1200C>G,STOP_GAINED,7,12_56004589_56004589_C_G
c.650G>A,MISSENSE_VARIANT,3,12_56004039_56004039_G_A
c.1096C>T,MISSENSE_VARIANT,3,12_56004485_56004485_C_T
c.1376G>A,MISSENSE_VARIANT,3,12_56004765_56004765_G_A
c.1521_1524del,FRAMESHIFT_VARIANT,2,12_56004905_56004909_ATTGT_A
c.1549_1574dup,FRAMESHIFT_VARIANT,2,12_56004933_56004933_A_ACAATGTGCAGCCAGACACCGTGGCCC
c.1382A>T,MISSENSE_VARIANT,2,12_56004771_56004771_A_T
c.884G>A,MISSENSE_VARIANT,2,12_56004273_56004273_G_A


Group and count the variants according to the predicted functional effect on the transcript of interest:

In [8]:
for ve, count in cohort.list_data_by_tx()[tx_id].items():
    print(f'{ve:<20}: {count}')

MISSENSE_VARIANT    : 29
STOP_GAINED         : 10
FRAMESHIFT_VARIANT  : 9


## Configure the analysis

We create an analysis runner, a convenience class for running the available analyses.

In [9]:
from gpsea.analysis import configure_cohort_analysis
from gpsea.analysis.predicate import BooleanPredicate

analysis = configure_cohort_analysis(cohort, hpo)

## Missense vs. other variants

Let's investigate correlation between missense variants vs. other variant effects.

The analysis runner prepares an *induced graph* from the HPO terms of the subjects. The induced graph is a `set` of the terms and their ancestors. 

> This is due to the annotation propagation rule of the ontologies with `is_a` relations (such as HPO) where presence of a term implies presence of its ancestors.

In [10]:
from gpsea.model import VariantEffect

result = analysis.compare_by_variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id)

Now we have `result` with the analysis results.

The runner tested each term of the induced graph for genotype-phenotype correlation. The patients are split into a 2x2 contingency table according to having the missense variant (or not) and being annotated with the HPO term (or not).

Let's look at one such table:

In [11]:
seizure = hpotk.TermId.from_curie('HP:0001250')
result.all_counts.loc[seizure]

MISSENSE_VARIANT on NM_001032386.2,No,Yes
category,,
No,0,7
Yes,11,17


We see that `24` (`7 + 17`) subjects had at least one missense variant while `11` had no missense variant.

Regarding presence of *Seizure*, we see that `28` (`11 + 17`) subjects were annotated with *Seizure* or one of its descendants, such as *Clonic seizure*, *Motor seizure*, etc.
`7` subjects were not annotated with any kind of a *Seizure*.

We can apply Fisher's exact test to test for significance and we will obtain the following p value:

In [12]:
print(f'p value: {result.pvals.loc[seizure]:.5f}')

p value: 0.07213


However, since we tested all features of the induced graph, we should apply multiple testing correction.

In [13]:
print(f'Ran {len(result.pvals)} tests in total')

Ran 68 tests in total


By default, we use *Bonferroni* correction:

In [14]:
print(f'After Bonferroni correction: {result.corrected_pvals[seizure]:.5f}')

After Bonferroni correction: 1.00000


We can summarize the results of all tests into a data frame. 
We use `BooleanPredicate.YES` to indicate that we want to see counts of patients *with* the tested phenotype.

In [15]:
result.summarize(hpo, BooleanPredicate.YES)

MISSENSE_VARIANT on NM_001032386.2                     No               Yes  \
                                                    Count    Percent  Count   
Seizure [HP:0001250]                                11/35  31.428571  17/35   
Hypouricemia [HP:0003537]                            4/15  26.666667   3/15   
Cognitive regression [HP:0034332]                    0/25   0.000000   6/25   
Increased urinary taurine [HP:0003166]                0/6   0.000000    5/6   
Hypotonia [HP:0001252]                               3/23  13.043478  12/23   
...                                                   ...        ...    ...   
Developmental regression [HP:0002376]                 0/6   0.000000    6/6   
Increased urine proteinogenic amino acid deriva...    0/5   0.000000    5/5   
Elevated circulating S-sulfocysteine concentrat...    0/2   0.000000    2/2   
Abnormal circulating non-proteinogenic amino ac...    0/2   0.000000    2/2   
Xanthinuria [HP:0010934]                             0/11   0.000000   2/11   

MISSENSE_VARIANT on NM_001032386.2                                        \
                                                       Percent   p value   
Seizure [HP:0001250]                                 48.571429  0.072129   
Hypouricemia [HP:0003537]                            20.000000  0.118881   
Cognitive regression [HP:0034332]                    24.000000  0.129170   
Increased urinary taurine [HP:0003166]               83.333333  0.166667   
Hypotonia [HP:0001252]                               52.173913  0.181896   
...                                                        ...       ...   
Developmental regression [HP:0002376]               100.000000  1.000000   
Increased urine proteinogenic amino acid deriva...  100.000000  1.000000   
Elevated circulating S-sulfocysteine concentrat...  100.000000  1.000000   
Abnormal circulating non-proteinogenic amino ac...  100.000000  1.000000   
Xanthinuria [HP:0010934]                             18.181818  1.000000   

MISSENSE_VARIANT on NM_001032386.2                                    
                                                   Corrected p value  
Seizure [HP:0001250]                                             1.0  
Hypouricemia [HP:0003537]                                        1.0  
Cognitive regression [HP:0034332]                                1.0  
Increased urinary taurine [HP:0003166]                           1.0  
Hypotonia [HP:0001252]                                           1.0  
...                                                              ...  
Developmental regression [HP:0002376]                            1.0  
Increased urine proteinogenic amino acid deriva...               1.0  
Elevated circulating S-sulfocysteine concentrat...               1.0  
Abnormal circulating non-proteinogenic amino ac...               1.0  
Xanthinuria [HP:0010934]                                         1.0  

[68 rows x 6 columns]

The summary shows patient counts and percentages along with corrected and uncorrected values. 
The rows are sorted by corrected and uncorrected values in ascending order.

## Task

We are seeking a smarter strategy to reduce the number of tests.

### Tune genotype predicate selection

First, we want the user to be well informed about the promising genotype predicates. For instance, it makes no sense to test missense variants vs. others if there are no missense variants in the gene.

We are developing code for exploratory analysis, including functions that plot variants on transcript and the protein, to inform the genotype predicate selection.

### Tune tested phenotypic features

Reducing the number of tested phenotypic features should increase the utility of the analysis.

One way to approach this is to leverage the ontology hierarchy to limit testing of the non-specific terms, such as *Abnormal nervous system physiology* (HP:0012638), which are unlikely to be helpful to the clinicians/users.

Next, we can select multiple testing correction methods that are more appropriate for working with ontology terms. We can probably combine this with an iterative approach, where we start testing the leaves of the induced graph (specific terms, such as *Focal clonic seizure*) and we "walk" the graph towards the general terms such as *Abnormal nervous system physiology*. The approach can include early termination if the current term exceeds a threshold (we need to define this better).

These are some ideas to start.

EOF